In [ ]:
import Ouzo_Sampling_Wtf_Lattice_Volume as create
import OT2_Ouzo_Commands as ot2_commands
import Ouzo_Graph_Tools as ouzo_graphs
from opentrons import simulate, execute, protocol_api
import pandas as pd
import numpy as np

In [ ]:
experiment_dict = create.get_experiment_plan(r"C:\Users\Edwin\Downloads\10_13_2020_E_P_Scan.csv")

In [ ]:
wtf_sample_canidates = create.generate_candidate_lattice_concentrations(experiment_dict) # there is a built in filter for anything summing to over 1, could be somewhat informative
volume_canidates = create.calculate_ouzo_volumes(wtf_sample_canidates, experiment_dict) # this is where you can iterate through a range of stocks and present different sets of possible volumes
filtered_wtf_samples, filtered_volume_samples = create.filter_samples(wtf_sample_canidates, volume_canidates, 30, 1000)
# print('Samples = ', len(filtered_wtf_samples))
# # print((sum(volume_canidates[0][0:4])/sum(volume_canidates[0]))*100)
# for volume in volume_canidates:
#     print(volume)

In [ ]:
# since autoscaled do not plot sample list with blank
ouzo_graphs.graph_canidates(experiment_dict, wtf_sample_canidates, filtered_wtf_samples) # add log/scinot, exclude blank by -1 on len

In [ ]:
# Optional addition of blank samples to sample volumes
average_vol = create.average_volume(filtered_volume_samples)
filtered_wtf_samples_b, filtered_volume_samples_b = create.add_blank(filtered_wtf_samples, filtered_volume_samples,average_vol, experiment_dict['Blank Component Concentrations'])

In [ ]:
protocol = simulate.get_protocol_api('2.0')
information = ot2_commands.run(protocol, experiment_dict, filtered_volume_samples_b)

In [ ]:
for i in information:
    print(i)

In [ ]:
create.create_csv(r"C:\Users\Edwin\Desktop\cmonwidg", information, filtered_wtf_samples_b, experiment_dict)
df = pd.read_csv(r"C:\Users\Edwin\Desktop\cmonwidg")

In [ ]:
df